In [1]:
import sys
sys.path.append('../')
from setting import config_read

In [2]:
import rdflib
import pandas as pd
import owlready2

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation




In [3]:
config = config_read('../')

# Load owl file
data_path = config['owl']['path']
onto = owlready2.get_ontology(data_path).load()

g = rdflib.Graph()
g.parse(data_path)

knows_query = """
SELECT DISTINCT ?x ?y ?z
WHERE {
    ?x ?y ?z.
}"""

triple_list = []
qres = g.query(knows_query)
for row in qres:
    triple_list.append([str(row[0]), str(row[1]), str(row[2])])
len(triple_list)

3825

In [4]:
triple_text_list = []
for s,p,o in triple_list:
    s_res = onto.search_one(iri=s)
    if s_res == None : 
        s_triple = s
    else:
        s_triple = s_res

    p_res = onto.search_one(iri=p)
    if p_res == None :
        p_triple = p
    else:
        p_triple = p_res

    o_res = onto.search_one(iri=o)
    if o_res == None : 
        o_triple = o
    else:
        o_triple = o_res
    
    triple_text_list.append([s_triple, p_triple, o_triple])

In [5]:
triple_df = pd.DataFrame(triple_text_list, columns=['S','P','O'])
print('겹치는 트리플 개수 : ', sum(triple_df.duplicated(['S','P','O'])))  
triple_df = triple_df.drop_duplicates(['S','P','O']).reset_index(drop=True) # 중복제거
print('중복 제거 후 트리플 개수 : ', len(triple_df))

겹치는 트리플 개수 :  157
중복 제거 후 트리플 개수 :  3668


In [6]:
p_list = list(onto.object_properties())
p_list.extend(list(onto.data_properties()))
only_op_dp_df = triple_df[triple_df.P.isin(p_list)].reset_index(drop=True)
only_op_dp_df

,S,P,O
0,SKMO_v2.4_0223.블랙핑크,SKMO_v2.4_0223.groupType,걸그룹
1,SKMO_v2.4_0223.빅뱅,SKMO_v2.4_0223.hasLeader,SKMO_v2.4_0223.지드래곤
2,SKMO_v2.4_0223.Bigbang,dc.title,Bigbang
3,SKMO_v2.4_0223.유리,foaf.gender,여성
4,SKMO_v2.4_0223.흔들어_feat_이은주,SKMO_v2.4_0223.hasGenre,SKMO_v2.4_0223.R&B
...,...,...,...
1000,SKMO_v2.4_0223.20160601_가온차트_1위,SKMO_v2.4_0223.rankDate,2016-07-06
1001,SKMO_v2.4_0223.지드래곤,foaf.birthday,2006-08-19
1002,SKMO_v2.4_0223.대한민국_연예예술상_시상식_여자_그룹상,SKMO_v2.4_0223.awardIn,SKMO_v2.4_0223.대한민국_연예예술상_시상식
1003,SKMO_v2.4_0223.웃어본다_대성_solo,SKMO_v2.4_0223.hasGenre,SKMO_v2.4_0223.hip_hop


In [7]:
def literal(x):
    if type(x) == str:
        return 'Literal_'
    else:
        return x.is_a

In [8]:
only_op_dp_df['domain'] = only_op_dp_df['S'].apply(lambda x:x.is_a)
only_op_dp_df['range'] = only_op_dp_df['O'].apply(literal)

only_op_dp_df = only_op_dp_df.explode(['domain'])
only_op_dp_df = only_op_dp_df.explode(['range'])

class_map_df = only_op_dp_df[only_op_dp_df['domain'] != owlready2.Thing].reset_index(drop=True).astype(str)

cond = class_map_df['range'] == 'Literal_'
class_map_df.loc[cond, 'range'] += class_map_df.loc[cond, 'P']

class_map_df

,S,P,O,domain,range
0,SKMO_v2.4_0223.블랙핑크,SKMO_v2.4_0223.groupType,걸그룹,mo.MusicGroup,Literal_SKMO_v2.4_0223.groupType
1,SKMO_v2.4_0223.빅뱅,SKMO_v2.4_0223.hasLeader,SKMO_v2.4_0223.지드래곤,mo.MusicGroup,SKMO_v2.4_0223.MusicArtist
2,SKMO_v2.4_0223.빅뱅,SKMO_v2.4_0223.hasLeader,SKMO_v2.4_0223.지드래곤,mo.MusicGroup,foaf.Person
3,SKMO_v2.4_0223.빅뱅,SKMO_v2.4_0223.hasLeader,SKMO_v2.4_0223.지드래곤,SKMO_v2.4_0223.MusicArtist,SKMO_v2.4_0223.MusicArtist
4,SKMO_v2.4_0223.빅뱅,SKMO_v2.4_0223.hasLeader,SKMO_v2.4_0223.지드래곤,SKMO_v2.4_0223.MusicArtist,foaf.Person
...,...,...,...,...,...
1422,SKMO_v2.4_0223.지드래곤,foaf.birthday,2006-08-19,SKMO_v2.4_0223.MusicArtist,Literal_foaf.birthday
1423,SKMO_v2.4_0223.지드래곤,foaf.birthday,2006-08-19,foaf.Person,Literal_foaf.birthday
1424,SKMO_v2.4_0223.대한민국_연예예술상_시상식_여자_그룹상,SKMO_v2.4_0223.awardIn,SKMO_v2.4_0223.대한민국_연예예술상_시상식,SKMO_v2.4_0223.Award,SKMO_v2.4_0223.Ceremony
1425,SKMO_v2.4_0223.웃어본다_대성_solo,SKMO_v2.4_0223.hasGenre,SKMO_v2.4_0223.hip_hop,mo.Track,SKMO_v2.4_0223.Genre


In [9]:
# the number of instance-level triples containing the property of the unit path
dpr_df = class_map_df.groupby(['domain', 'P', 'range'], as_index=False).S.count()

# the total number of triples from domain class to range class
dr_df = class_map_df.groupby(['domain', 'range'], as_index=False).S.count()

merged_df = pd.merge(class_map_df, dpr_df, on=['domain', 'P', 'range'], how='inner', suffixes=('','_dpr'))
merged_df = pd.merge(merged_df, dr_df, on=['domain', 'range'], how='inner', suffixes=('','_dr'))
merged_df['W'] = 1 - (merged_df['S_dpr'] / merged_df['S_dr'])

final_triple_df = merged_df[['domain', 'P', 'range', 'W']]
merged_df = merged_df[['S', 'P', 'O', 'domain', 'range']]
final_triple_df = final_triple_df.drop_duplicates().reset_index(drop=True)

In [10]:
final_triple_df

,domain,P,range,W
0,mo.MusicGroup,SKMO_v2.4_0223.groupType,Literal_SKMO_v2.4_0223.groupType,0.00
1,mo.MusicGroup,SKMO_v2.4_0223.hasLeader,SKMO_v2.4_0223.MusicArtist,0.25
2,mo.MusicGroup,foaf.member,SKMO_v2.4_0223.MusicArtist,0.75
3,mo.MusicGroup,SKMO_v2.4_0223.hasLeader,foaf.Person,0.20
4,mo.MusicGroup,foaf.member,foaf.Person,0.80
...,...,...,...,...
103,mo.MusicGroup,SKMO_v2.4_0223.debutTrack,mo.Track,0.00
104,SKMO_v2.4_0223.MusicArtist,SKMO_v2.4_0223.debutDate,Literal_SKMO_v2.4_0223.debutDate,0.00
105,mo.Track,SKMO_v2.4_0223.lyrics,Literal_SKMO_v2.4_0223.lyrics,0.00
106,SKMO_v2.4_0223.Instrumental,schema.datePublished,Literal_schema.datePublished,0.00


In [11]:
merged_df.to_csv('../unit_path_for_mapping.csv', index=False)
final_triple_df.to_csv('../unit_path.csv', index=False)